In [1]:
import pandas as pd
import os
import csv
import glob
import numpy as np
import collections
import matplotlib.pyplot as plt

Prices = collections.namedtuple('Prices', field_names=['open', 'high', 'low', 'close', 'volume'])

In [2]:
train_path = "D:\Practical_Reinforcement_Learning\chapter8\data\ch08-small-quotes\YNDX_150101_151231.csv"

In [3]:
df = pd.read_csv(os.path.abspath(train_path))

index = ['<OPEN>', "<HIGH>", "<LOW>","<CLOSE>","<VOL>"]
df[index] = df[index].astype(float)
df_normalized = (df - df.min()) / (df.max() - df.min())
# Define the tolerance value
tolerance = 1e-8

# Apply the lambda function to check if each value is within the tolerance of the first value
#result = df_normalized.applymap(lambda v: abs(v - df_normalized.iloc[0]) < tolerance)
df

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,20150105,100100,1064.0,1064.0,1064.0,1064.0,0.0
1,20150105,100200,1064.0,1064.0,1064.0,1064.0,0.0
2,20150105,100300,1064.0,1064.0,1064.0,1064.0,0.0
3,20150105,100400,1064.0,1064.0,1064.0,1064.0,0.0
4,20150105,100500,1064.0,1064.0,1064.0,1064.0,0.0
...,...,...,...,...,...,...,...
130561,20151230,183800,1146.8,1146.8,1146.3,1146.3,100.0
130562,20151230,183900,1146.8,1146.8,1146.8,1146.8,0.0
130563,20151230,184000,1146.8,1146.8,1146.8,1146.8,9.0
130564,20151230,184600,1147.0,1147.0,1147.0,1147.0,520.0


In [4]:
prev_vals = None
fix_open_price  = True
o, h, l, c, v = [], [], [], [], []
count_out = 0
count_filter = 0
count_fixed = 0
for row in df.itertuples():
    val = (row._3,row._4,row._5,row._6,row._7)
    po, ph, pl,pc,pv = val
    if fix_open_price and prev_vals is not None:
        ppo, pph, ppl, ppc, ppv = prev_vals
        if abs(po - ppc) > 1e-8:
            count_fixed += 1
            po = ppc
            pl = min(pl, po)
            ph = max(ph, po)
            count_out += 1
    o.append(po)
    c.append(pc)
    h.append(ph)
    l.append(pl)
    v.append(pv)
    prev_vals = val

In [5]:
print(o)

[1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1064.0, 1053.0, 1051.0, 1055.0, 1055.0, 1055.0, 1055.0, 1055.0, 1055.0, 1055.0, 1055.0, 1055.0, 1055.0, 1055.0, 1055.0, 1056.0, 1056.0, 1065.0, 1066.0, 1065.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1066.0, 1066.0, 1066.0, 1066.0, 1066.0, 1066.0, 1066.0, 1066.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1066.0, 1066.0, 1066.0, 1065.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1066.0, 1066.0, 1066.0, 1066.0, 1066.0, 1066.0, 1066.0, 1066.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1067.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1065.0, 1066.0,

In [6]:
prices=Prices(open=np.array(o, dtype=np.float32),
                  high=np.array(h, dtype=np.float32),
                  low=np.array(l, dtype=np.float32),
                  close=np.array(c, dtype=np.float32),
                  volume=np.array(v, dtype=np.float32))

In [7]:
def prices_to_relative(prices):
    """
    Convert prices to relative in respect to open price
    :param ochl: tuple with open, close, high, low
    :return: tuple with open, rel_close, rel_high, rel_low
    """
    assert isinstance(prices, Prices)
    rh = (prices.high - prices.open) / prices.open
    rl = (prices.low - prices.open) / prices.open
    rc = (prices.close - prices.open) / prices.open
    return Prices(open=prices.open, high=rh, low=rl, close=rc, volume=prices.volume)

relative_price = prices_to_relative(prices)

In [8]:
relative_price

Prices(open=array([1064. , 1064. , 1064. , ..., 1146.8, 1146.8, 1147. ], dtype=float32), high=array([0.        , 0.        , 0.        , ..., 0.        , 0.00017436,
       0.        ], dtype=float32), low=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), close=array([0.        , 0.        , 0.        , ..., 0.        , 0.00017436,
       0.        ], dtype=float32), volume=array([  0.,   0.,   0., ...,   9., 520.,   2.], dtype=float32))

In [11]:
import gym
import gym.spaces
from gym.utils import seeding
from gym.envs.registration import EnvSpec
import enum
import numpy as np


DEFAULT_BARS_COUNT = 10
DEFAULT_COMMISSION_PERC = 0.1



class Actions(enum.Enum):
    Skip = 0
    Buy = 1
    Close = 2

class State:
    def __init__(self, bars_count, commission_perc,
                 reset_on_close, reward_on_close=True,
                 volumes=True):
        assert isinstance(bars_count, int)
        assert bars_count > 0
        assert isinstance(commission_perc, float)
        assert commission_perc >= 0.0
        assert isinstance(reset_on_close, bool)
        assert isinstance(reward_on_close, bool)
        self.bars_count = bars_count
        self.commission_perc = commission_perc
        self.reset_on_close = reset_on_close
        self.reward_on_close = reward_on_close
        self.volumes = volumes

    def reset(self, prices, offset):
        assert isinstance(prices, Prices)
        assert offset >= self.bars_count-1
        self.have_position = False
        self.open_price = 0.0
        self._prices = prices
        self._offset = offset

    @property
    def shape(self):
        # [h, l, c] * bars + position_flag + rel_profit
        if self.volumes:
            return 4 * self.bars_count + 1 + 1,
        else:
            return 3*self.bars_count + 1 + 1,

    def encode(self):
        """
        Convert current state into numpy array.
        """
        res = np.ndarray(shape=self.shape, dtype=np.float32)
        shift = 0
        for bar_idx in range(-self.bars_count+1, 1):
            ofs = self._offset + bar_idx
            
            res[shift] = self._prices.high[ofs]
            shift += 1
            res[shift] = self._prices.low[ofs]
            shift += 1
            res[shift] = self._prices.close[ofs]
            shift += 1
            if self.volumes:
                res[shift] = self._prices.volume[ofs]
                shift += 1
            print(f"""state off set ofs {ofs}\n and shape res as batch from offset {res.shape} \n 
                    state_offset {self._offset} \n bar_idx {bar_idx} \n shift {shift} \n res shift {res}""")
        res[shift] = float(self.have_position)
        shift += 1
        if not self.have_position:
            res[shift] = 0.0
        else:
            res[shift] = self._cur_close() / self.open_price - 1.0
        print(f"Final res shape {res.shape} shift {shift}")
        return res

    def _cur_close(self):
        """
        Calculate real close price for the current bar
        """
        open = self._prices.open[self._offset]
        rel_close = self._prices.close[self._offset]
        return open * (1.0 + rel_close)

    def step(self, action):
        """
        Perform one step in our price, adjust offset, check for the end of prices
        and handle position change
        :param action:
        :return: reward, done
        """
        assert isinstance(action, Actions)
        reward = 0.0
        done = False
        close = self._cur_close()
        if action == Actions.Buy and not self.have_position:
            self.have_position = True
            self.open_price = close
            reward -= self.commission_perc
        elif action == Actions.Close and self.have_position:
            reward -= self.commission_perc
            done |= self.reset_on_close
            if self.reward_on_close:
                reward += 100.0 * (close / self.open_price - 1.0)
            self.have_position = False
            self.open_price = 0.0

        self._offset += 1
        prev_close = close
        close = self._cur_close()
        done |= self._offset >= self._prices.close.shape[0]-1

        if self.have_position and not self.reward_on_close:
            reward += 100.0 * (close / prev_close - 1.0)

        return reward, done


class StocksEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    #spec = EnvSpec("StocksEnv-v0",entry_point=libs.envoiran.StocksEnv)

    def __init__(self, prices, bars_count=DEFAULT_BARS_COUNT,
                 commission=DEFAULT_COMMISSION_PERC,
                 reset_on_close=True, state_1d=False,
                 random_ofs_on_reset=True, reward_on_close=False,
                 volumes=False):
        self._prices = prices
        self._state = State(
            bars_count, commission, reset_on_close,
            reward_on_close=reward_on_close, volumes=volumes)
        self.action_space = gym.spaces.Discrete(n=len(Actions))
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf,
            shape=self._state.shape, dtype=np.float32)
        self.random_ofs_on_reset = random_ofs_on_reset
        
        #self.seed()
    def seed(self, seed=None):
        self.np_random, seed1 = seeding.np_random(seed)
        seed2 = seeding.hash_seed(seed1 + 1) % 2 ** 31
        return [seed1, seed2]
    
    def reset(self):
        self._instrument = self.np_random.choice(
            list(self._prices._fields))
        if self._instrument is "open":
            prices = self._prices.open
        if self._instrument is "close":
            prices = self._prices.close
        if self._instrument is "high":
            prices = self._prices.high
        if self._instrument is "low":
            prices = self._prices.low
        else:
            prices = self._prices.volume
        bars = self._state.bars_count
        if self.random_ofs_on_reset:
            offset = self.np_random.choice(
                prices.shape[0]-bars*10) + bars
        else:
            offset = bars
        print(self._prices.low[offset],offset)
        
        # return P, offset
        self._state.reset(self._prices, offset)
        return self._state.encode()
    def step(self, action_idx):
        action = Actions(action_idx)
        reward, done = self._state.step(action)
        obs = self._state.encode()
        info = {

            
                "instrument": self._instrument,
                "offset": self._state._offset
                }
        return obs, reward, done, info

    


<>:150: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:152: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:154: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:156: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:150: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:152: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:154: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:156: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\sushant\AppData\Local\Temp\ipykernel_21064\3350157000.py:150: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._instrument is "open":
C:\Users\sushant\AppData\Local\Temp\ipykernel_21064\3350157000.py:152: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._instrument is "close":
C:\Users\sushant\AppData\Local\Temp\ipykernel_21064\3350157000.py:154: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._instrument is "high":
C:\Users\sushant\A

In [12]:

env  = StocksEnv(relative_price, bars_count=10,
                 commission=0.1,
                 reset_on_close=True, state_1d=False,
                 random_ofs_on_reset=True, reward_on_close=False,
                 volumes=True)

In [13]:
obs=env.reset()

0.0 123544
state off set ofs 123535
 and shape res as batch from offset (42,) 
 
                    state_offset 123544 
 bar_idx -9 
 shift 4 
 res shift [ 0.  0.  0.  0. inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf]
state off set ofs 123536
 and shape res as batch from offset (42,) 
 
                    state_offset 123544 
 bar_idx -8 
 shift 8 
 res shift [ 0.  0.  0.  0.  0.  0.  0.  0. inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf]
state off set ofs 123537
 and shape res as batch from offset (42,) 
 
                    state_offset 123544 
 bar_idx -7 
 shift 12 
 res shift [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf]
state off set ofs 123538
 and shape r

In [14]:
print(obs)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [16]:
obs, reward, done, info= env.step(1)

state off set ofs 123537
 and shape res as batch from offset (42,) 
 
                    state_offset 123546 
 bar_idx -9 
 shift 4 
 res shift [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
state off set ofs 123538
 and shape res as batch from offset (42,) 
 
                    state_offset 123546 
 bar_idx -8 
 shift 8 
 res shift [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
state off set ofs 123539
 and shape res as batch from offset (42,) 
 
                    state_offset 123546 
 bar_idx -7 
 shift 12 
 res shift [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
state off set ofs 123540
 and shape res as batch from offset (42,) 
 
                    state_offset 123546 
 bar_idx -6 
 shift 16 
 res shift [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [17]:
print(obs, reward, done, info)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] 0.0 False {'instrument': 'close', 'offset': 123546}


All done for env .
lets setup model to get q value from our observation.

state off set ofs 113709
 and shape res as batch from offset (42,) 
 
                    state_offset 113718 
 bar_idx -9 
 shift 4 
 res shift [ 1.9666884e-04  0.0000000e+00  1.9666884e-04  4.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  4.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  4.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  4.0000000e+00
  0.0000000e+00 -1.9663016e-04 -1.9663016e-04  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  2.0000000e+00
  1.9666884e-04  0.0000000e+00  1.9666884e-04  0.0000000e+00
  0.0000000e+00  0.0000000e+00]
state off set ofs 113710
 and shape res as batch from offset (42,) 
 
                    state_offset 113718 
 bar_idx -8 
 shift 8 
 res shift [ 1.9666884e-04  0.0000000e+00  1.9666884e-04  4.0000000e+00
  0.000

[  0.   0.   0. ...   9. 520.   2.] 69610
